<a href="https://colab.research.google.com/github/WRFitch/fyp/blob/main/src/fyp_ai_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AI data analysis

### Setup
- Install & import necessary libraries
- Mount drive
- Import and define handy variables 

In [ ]:
# Sometimes the colab fastai version can be wrong, so we reinstall with no cache
# uninstalling, reinstalling, and restarting runtime should fix any major issues. 
!pip uninstall -y fastai
!pip install -U --no-cache-dir fastai

In [ ]:
# TODO clean up imports - I can't seriously need all this
import os
import numpy as np
import pandas as pd

from fastai import *
from fastai.callback.hook import * 
from fastai.tabular import *
from fastai.vision import *
from fastai.vision.all import *
from google.colab import drive
import matplotlib.pyplot as plt

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%rm -rf /content/fyp

In [ ]:
%cd /content
!git clone https://github.com/WRFitch/fyp.git

/content
Cloning into 'fyp'...
remote: Enumerating objects: 28, done.
remote: Counting objects: 100% (28/28), done.
remote: Compressing objects: 100% (28/28), done.
remote: Total 999 (delta 15), reused 0 (delta 0), pack-reused 971
Receiving objects: 100% (999/999), 166.68 MiB | 39.42 MiB/s, done.
Resolving deltas: 100% (585/585), done.


In [ ]:
# Import fyputil library
%cd /content/fyp/src/fyputil
import constants as c
import fyp_utils as fyputil
%cd /content

/content/fyp/src/fyputil
/content


In [ ]:
drive_path = "/content/drive/MyDrive/"
png_dir = c.png_dir
model_dir = c.model_dir
tfm_dir = f"{c.data_dir}/png_tfms"
big_png_dir = f"{c.data_dir}/png_224"
big_tfm_dir = f"{c.data_dir}/224_png_tfms"

#### Test Image Import

(Not strictly necessary, but nice to have) 

In [ ]:
# Import data from google drive 
# This is getting really slow. Is there too much data? If so, slice to only use 
# every tenth image so we still get a decently stratified set. 
#imgs = get_image_files(c.png_dir)
#print(len(imgs))

In [ ]:
#big_imgs = get_image_files(big_png_dir)
#print(len(big_imgs))

In [ ]:
#tfm_imgs = get_image_files(tfm_dir)
#print(len(tfm_imgs))

In [ ]:
big_tfms = get_image_files(big_tfm_dir)
len(big_tfms)

### Data Setup

#### Sort Dataframes

In [ ]:
# Parse CSVs into pandas dataframes
# TODO rewrite so we aren't deleting columns directly - do it properly! Incorporate these into one csv export in the 
#      output pipeline 
co_df = pd.read_csv(f"{c.data_dir}/{c.CO_band}.csv")
del co_df[".geo"]
hcho_df = pd.read_csv(f"{c.data_dir}/{c.HCHO_band}.csv")
del hcho_df[".geo"]
no2_df = pd.read_csv(f"{c.data_dir}/{c.NO2_band}.csv")
del no2_df[".geo"]
o3_df = pd.read_csv(f"{c.data_dir}/{c.O3_band}.csv")
del o3_df[".geo"]
so2_df = pd.read_csv(f"{c.data_dir}/{c.SO2_band}.csv")
del so2_df[".geo"]
ch4_df = pd.read_csv(f"{c.data_dir}/{c.CH4_band}.csv")
del ch4_df[".geo"]

In [ ]:
# Incorporate individual csvs into one ghg dataframe. Badly. 
# TODO fix this so we aren't repeating the same thing over and over
mrg_params = [c.lon, c.lat, 'system:index']
# somehow this means "intersect". We're taking the intersect so we know we have common values. 
mrg_type = 'inner'

intersect = pd.merge(so2_df, ch4_df, how=mrg_type, on=mrg_params)
intersect = pd.merge(intersect, co_df, how=mrg_type, on=mrg_params)
intersect = pd.merge(intersect, hcho_df, how=mrg_type, on=mrg_params)
intersect = pd.merge(intersect, no2_df, how=mrg_type, on=mrg_params)
intersect = pd.merge(intersect, o3_df, how=mrg_type, on=mrg_params)

print(intersect.shape)
intersect.iloc[0:4] 

In [ ]:
raw_ghg_df = intersect.copy()

for index, row in intersect.iterrows():
  coords = (row.longitude, row.latitude)
  #print(coords)
  if not fyputil.imgExported(coords):
    #print(f"dropping {fyputil.getFilepath(coords)} from row {index}")
    # TODO implement this in a way that doesn't recreate the dataframe thousands of times
    raw_ghg_df = raw_ghg_df.drop(index=index)

# Normalise ghg numbers so they can be more easily predicted by the network, and 
# fit into a consistent, normalised y_range 
ghg_df = raw_ghg_df.copy()
ghg_df = fyputil.normGhgDfProperly(ghg_df)

print(intersect.shape)
print(raw_ghg_df.shape)
raw_ghg_df.iloc[0:10]

In [ ]:
ghg_df

In [ ]:
ghg_df.to_csv(f"{c.data_dir}/good_ghg.csv")

In [ ]:
def copyUsableImages():
  for idx, row in ghg_df.iterrows():
    coords = (row.longitude, row.latitude)
    filename = fyputil.getFilepath(coords)
    os.system(f"cp {filename} {c.data_dir}/png_tfms/orig/normal/{row.longitude}_{row.latitude}")
    print(idx)

#### Sort dataframes in a way that's far easier
THIS IS THE ONLY PART THAT'S NECESSARY FOR THE CURRENT TRAINING PIPELINE

In [ ]:
ghg_df = pd.read_csv(c.ghg_csv)
ghg_df = fyputil.normGhgDfProperly(ghg_df)

#### Sort Data Import Methods

In [ ]:
def getGhgsAsArr(img_path):
  return fyputil.getGhgsAsArr(img_path, ghg_df)

#def imgIsInDf(img_path):
#  # Probably a faster way to do this
#  val = fyputil.getGhgs(img_path, ghg_df)
#  if val == None: return False 
#  if len(val) == 6: return True 
#  return False

def getGhgImgs(path):
  return get_image_files(path).filter(fyputil.imgIsInDf)

In [ ]:
# Look for any duplicated files and remove 
count = 0
for root, dir, files in os.walk(big_tfm_dir):
  for file in files:
    if "(" in file:
      print(f"{root}/{file}")
      os.remove(f"{root}/{file}")
      count += 1
print(count)

### Create DataBlocks for training the model 

#### Standard GHG Block


In [ ]:
# TODO implement multiple transforms pipeline
# TODO revisit image normalisation

ghg_block = DataBlock(
    blocks = (ImageBlock, RegressionBlock),
    get_items = getGhgImgs,
    get_y = getGhgsAsArr,
    item_tfms = Resize(224)
)

ghg_dl = ghg_block.dataloaders(big_png_dir)

In [ ]:
ghg_dl.show_batch(nrows=9, max_n=9, figsize = (50,50))

In [ ]:
ghg_block.summary(big_png_dir)

## Training

### Image Recognition and Feature Extraction. 

- Train image-based predictor to guess greenhouse gas concentrations based on 1km square of land. 
  - Transfer an ImageNet predictor to work top-down
  - Start by predicting one ghg and expand from there
- Use image predictor to extract a basic feature set by slicing the network at different points. The idea is to limit the amount of data going into the tabular recommender, while transferring as much useful data as possible. We want to implicitly extract GHG-emitting features of each image without losing any detail, as a form of convolutional preprocessing. 


In [ ]:
# uses a regression approach.
# TODO analyse metrics. Really it doesn't seem to matter so long as everything 
#      is evaluated equally, but I'd like to be sure - ask Allan on Monday. 
# TODO Further experimentation with resnet size is necessary. 34 provides _ok_ 
#      predictions, longer is usually better but it takes longer to train. While
#      I'm iterating on design, performance is necessary. Once I'm at a stage 
#      where I can export my model and use it as is, I'll take the time to train 
#      a much larger network. 
learn = cnn_learner(ghg_dl, resnet152, y_range=(0, 100),  metrics=rmse)

In [ ]:
name = "learner test"
learn.save(name)

Path('models/learner test.pth')

In [ ]:
learn.load(name)

In [ ]:
# TODO examine 3d representation of problem space re: local optima 
learn.lr_find()

In [ ]:
lr = 0.05
finelr = 0.0019
xfinelr = 0.0001
xxfinelr = 2e-5

In [ ]:
# epochs = 5
learn.fine_tune(2, lr)

In [ ]:
# Saving mid-training, so I can figure out a decent training pathway
learn.save("mid-training")

Path('models/mid-training.pth')

In [ ]:
learn.load("mid-training")

In [ ]:
learn.fine_tune(5, finelr)

In [ ]:
learn.save("fine-tuning")

Path('models/fine-tuning.pth')

In [ ]:
learn.fine_tune(5, xfinelr)

In [ ]:
learn.save("xfine-tuning")

Path('models/xfine-tuning.pth')

In [ ]:
learn.load("xfine-tuning")

In [ ]:
# at this point, it doesn't seem to make any difference 
# There appears to be a point of diminishing returns, where rmse is just the 
# error rate of the given data. 
learn.fine_tune(10, xxfinelr)

## Evaluate Model Performance 

### Plot results 

In [ ]:
learn.show_results(ds_idx=1, dl=ghg_dl, nrows=9, max_n=9, figsize = (50,50))

In [ ]:
import fastai.utils.collect_env
fastai.utils.collect_env.show_install(1)

In [ ]:
interp = Interpretation.from_learner(learn)

### Plot model statistics 

#### Plot layer stats

This allows us to see what the mean std and pct activation levels are, letting us see areas of the network that require further analysis 

In [ ]:
learn.activation_stats.plot_layer_stats(151) 

In [ ]:
learn.recorder.plot_sched()

In [ ]:
learn.activation_stats.color_dim(-4)

### Export the model

In [ ]:
# Export model so we can use it for other things
# Note - this kills the model 
#TODO find better naming convention 
new_model = "mrghg_230221_resnet34"
learn.export(f"{c.model_dir}/{new_model}.pkl")

In [ ]:
# Import model and test to see if it hasn't broken in the export process.
imported_learner = load_learner(f"{c.model_dir}/{c.model_name}.pkl")

In [ ]:
# Predict from imported learner
imported_learner.predict(f"{c.png_dir}/-0.73212695655741_51.2533785354393.png")

((3.3061301708221436,
  6.372119903564453,
  5.765547275543213,
  1.3745146989822388,
  3.039674997329712,
  1.802012324333191),
 tensor([3.3061, 6.3721, 5.7655, 1.3745, 3.0397, 1.8020]),
 tensor([3.3061, 6.3721, 5.7655, 1.3745, 3.0397, 1.8020]))

#### Notes on Image Predictions

A lower learning rate appears to cause slower training with more sophisticated conclusions. Sophistication also appears to arise from a deeper network, but I'm hitting a wall at roughly 0.6 rmse.

---

Currently, the networks are having some trouble defining more subtle characteristics of the images, which shows some flaws in my work. The network will need some supplemental information to accurately predict the greenhouse gas at this point. This may include the following:
- **Latitude/Longitude.** Geography may affect predictions - all the images in my current dataset are near London, meaning they have far more greenhouse gases than most places. To encode a knowledge of city geography into a neural net may take some work...
- **Property Value.** How valuable is this land? This could go some way to encoding city dynamics, as well as explaining where the land might be. If land is rural, but valuable, it's likely to be near major cities or airports. 
- **Nearby GHG Values.** Combined with wind direction, an understanding of source & direction of airflow may describe how areas inherit ghg's from elsewhere. An example of this would be the high concentration of NO<sub>2</sub> north of Heathrow Airport, which may be caused by common flight patterns heading north. 
- **Wind Direction.** See above. 
- **Land Use.** Depending on detail, this may help alleviate the "grey field/massive factory" issue described in my log. By proving that certain areas are rural, residential, or industrial, we can limit errors based on inferring purely visual information. If we can specifically define what a large grey box is doing, we can also come to more developed conclusions about its purpose. A recycling center, an oil refinery, and a brewery may all look similar from above, but information about what they _are_ will limit a neural network getting confused. 
- **Population Density/Economic Output.** This will work in a similar way to property value, where we can predict human activity and its effects on greenhouse gases. Economic output may have a complex relationship to GHG emissions that cannot be easily represented, depending on the form of industry. For example, an eco-tourist attraction may rely on its low carbon footprint for survival, whereas a petrol station relies on high carbon ouput. 
- **Land Height**

Effectively, this network recognises certain features of high-GHG land. Depending on sophistication, this may include airports, power plants, or other rare features, as well as recognising different types of wilderness or residential districts. This will be used to extract a feature set for a tabular recommender, which can then be used to find more accurate readings. 